In [1]:
import pandas as pd
import numpy as np

from influxdb_client import InfluxDBClient, Point, Dialect

import re
import time
import datetime

import warnings
from influxdb_client.client.warnings import MissingPivotFunction

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.colors as colors

import pandasql as ps
import sqlite3

import csv

import warnings 
from influxdb_client.client.warnings import MissingPivotFunction
warnings.simplefilter("ignore", MissingPivotFunction)

import gc

pd.set_option('display.max_rows', 500)

In [2]:
def _parse_line(line):

    rx_dict = {
    'token': re.compile(r'var token = "(?P<token>.*)"\n'),
    'url': re.compile(r'var url = "(?P<url>.*)"\n'),
    'org': re.compile(r'var org = "(?P<org>.*)"\n'),
    'bucket': re.compile(r'var bucket = "(?P<bucket>.*)"\n'),
    }   

    """
    Do a regex search against all defined regexes and
    return the key and match result of the first matching regex

    """
    for key, rx in rx_dict.items():
        match = rx.search(line)
        if match:
            return key, match
    # if there are no matches
    return None, None

filepath = '/root/flexi-pipe/config.go'
# open the file and read through it line by line
with open(filepath, 'r') as file_object:
    line = file_object.readline()
    while line:
        # at each line check for a match with a regex
        key, match = _parse_line(line)

        if key == 'token':
            token = match.group('token')
        elif key == 'url':
            url = match.group('url')
        elif key == 'org':
            org = match.group('org')
        elif key == 'bucket':
            bucket = match.group('bucket')
        
        line = file_object.readline()
# url="http://192.168.20.58:8086"
url = "http://localhost:8086"

In [3]:
start = 1696445523
end = 1696710919

In [12]:
def experiment(start_time, end_time, filepath):
    # Retrieve experiments data from csv
    data = pd.read_csv(filepath, header=None)
    df = pd.DataFrame(data)

    #Rename columns
    experiments = df.rename(columns={0: "start", 1: "end", 2: "topology", 3: "runtime", 4: "parameter", 5: "d", 6: "dlo", 7: "dhi", 8: "dscore", 9: "dlazy", 10: "dout", 11: "gossipFactor", 12: "initialDelay", 13: "interval"}, errors='raise')

    #Correct timestamp
    experiments["start"] = experiments["start"].str.slice(0, 27)
    experiments["end"] = experiments["end"].str.slice(0, 27)

    #String to timestamp
    # experiments['startUnix'] = pd.to_datetime(experiments["start"],format="%Y-%m-%d %H:%M:%S.%f").astype('int64') / 10**9
    # experiments['endUnix'] = pd.to_datetime(experiments["end"],format="%Y-%m-%d %H:%M:%S.%f").astype('int64') / 10**9
    experiments['startUnix'] = pd.to_datetime(experiments["start"],format="mixed").astype('int64') / 10**9
    experiments['endUnix'] = pd.to_datetime(experiments["end"],format="mixed").astype('int64') / 10**9


    experiments['startUnix'] = pd.to_timedelta(experiments['startUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)
    experiments['endUnix'] = pd.to_timedelta(experiments['endUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)

    #Drop fields we don't mneed for the moment
    exp = experiments.drop(columns=["runtime", "initialDelay"]).sort_values(by=["start"])

    #Get times for different intervals
    # intervals = exp["interval"].drop_duplicates().sort_values().reset_index(drop=True)
    # intervals.head(10)

    # expTime = exp[exp['startUnix'].astype(int).between(start_time, end_time)]
    # expTime['experiment'] = expTime.index
    expTime = exp.reset_index().rename({'index':'experiment'}, axis = 'columns')

    return expTime

experiments = experiment(start, end, '../experiments.csv')
experiments.tail(150)

# data = pd.read_csv('../experiments.csv', header=0,  index_col=0)
# exps = pd.DataFrame(data)
#experiments = exps.loc[exps['experiment'] > 144]

,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix
516,516,2023-11-08 00:21:08.1585819,2023-11-08 00:51:08.2080839,unl,informed2,16,8,20,16,8,4,0.50,0.50,1699402868,1699404668
517,517,2023-11-08 00:54:37.1934314,2023-11-08 01:24:37.1980710,unl,informed2,16,8,20,16,8,4,0.50,0.50,1699404877,1699406677
518,518,2023-11-08 01:28:06.5063790,2023-11-08 01:58:06.5154308,unl,informed2,16,8,20,16,8,4,0.50,0.50,1699406886,1699408686
519,519,2023-11-08 02:01:36.1265616,2023-11-08 02:31:36.1312701,unl,informed3,6,4,8,2,2,2,0.25,1.00,1699408896,1699410696
520,520,2023-11-08 02:35:05.9104270,2023-11-08 03:05:05.9118565,unl,informed3,6,4,8,2,2,2,0.25,1.00,1699410905,1699412705
521,521,2023-11-08 03:08:36.1469516,2023-11-08 03:38:36.1486421,unl,informed3,6,4,8,2,2,2,0.25,1.00,1699412916,1699414716
522,522,2023-11-08 03:42:06.6471529,2023-11-08 04:12:06.6486114,unl,informed4,6,4,8,2,2,4,0.25,3.00,1699414926,1699416726
523,523,2023-11-08 04:15:36.2400420,2023-11-08 04:45:36.2499079,unl,informed4,6,4,8,2,2,4,0.25,3.00,1699416936,1699418736
524,524,2023-11-08 04:49:07.4268152,2023-11-08 05:19:07.4358495,unl,informed4,6,4,8,2,2,4,4.25,3.00,1699418947,1699420747
525,525,2023-11-08 05:22:38.8338722,2023-11-08 05:52:38.8574236,unl,informed5,6,4,8,2,2,4,0.25,0.50,1699420958,1699422758


In [45]:
def from_influx(url, token, org, measurement, start_time, end_time,grouping_key):
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)

    # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()

    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                                        ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                                        ' |> filter(fn: (r) => r._measurement == "'+measurement+'") '
                                        ' |> group(columns: ["_measurement", "_field"], mode: "by") '
                                        ' |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')
    client.close()

    # df = data_frame.drop(columns=['result', 'table','_start', '_stop', '_measurement', 'topic', 'receivedFrom']).sort_values(by=["_time"]).reset_index(drop=True)
    data_frame.reset_index(inplace=True)
    df = data_frame[['_time', grouping_key]].sort_values(by=["_time"]).reset_index(drop=True)
    df["_time"] = pd.to_datetime(df["_time"])

    return df

In [46]:
data = pd.read_csv('./experiments_filtered.csv', header=0,  index_col=0)
exps = pd.DataFrame(data)

exps.head(10)

,experiment,originalStart,originalEnd,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,start,end,startUnix,endUnix
3,390,2023-10-04 18:52:03.080773000,2023-10-04 19:22:03.086651800,unl,reference,8,6,12,4,8,2,0.25,1.0,2023-10-04 18:59:33.080773000,2023-10-04 19:14:33.086651800,1696445973,1696446873
4,391,2023-10-04 19:25:37.153339900,2023-10-04 19:55:37.155585600,unl,reference,8,6,12,4,8,2,0.25,1.0,2023-10-04 19:33:07.153339900,2023-10-04 19:48:07.155585600,1696447987,1696448887
6,393,2023-10-04 20:32:47.309861600,2023-10-04 21:02:47.314917800,unl,informed,8,6,12,4,8,2,0.25,3.0,2023-10-04 20:40:17.309861600,2023-10-04 20:55:17.314917800,1696452017,1696452917
7,394,2023-10-04 21:06:22.775563400,2023-10-04 21:36:22.778880000,unl,informed,8,6,12,4,8,2,0.25,3.0,2023-10-04 21:13:52.775563400,2023-10-04 21:28:52.778880000,1696454032,1696454932
9,396,2023-10-04 22:13:33.814670400,2023-10-04 22:43:33.817409400,unl,informed0,16,8,20,16,8,4,0.50,1.0,2023-10-04 22:21:03.814670400,2023-10-04 22:36:03.817409400,1696458063,1696458963
10,397,2023-10-04 22:47:09.383711000,2023-10-04 23:17:09.386844200,unl,informed0,16,8,20,16,8,4,0.50,1.0,2023-10-04 22:54:39.383711000,2023-10-04 23:09:39.386844200,1696460079,1696460979
15,404,2023-10-05 02:42:12.104487300,2023-10-05 03:12:12.156487200,unl,informed2,16,8,20,16,8,4,0.50,0.5,2023-10-05 02:49:42.104487300,2023-10-05 03:04:42.156487200,1696474182,1696475082
16,405,2023-10-05 03:15:44.641811200,2023-10-05 03:45:44.644068700,unl,informed3,6,4,8,2,2,2,0.25,1.0,2023-10-05 03:23:14.641811200,2023-10-05 03:38:14.644068700,1696476194,1696477094
17,406,2023-10-05 03:49:18.120429500,2023-10-05 04:19:18.122983600,unl,informed3,6,4,8,2,2,2,0.25,1.0,2023-10-05 03:56:48.120429500,2023-10-05 04:11:48.122983600,1696478208,1696479108
21,412,2023-10-05 07:10:41.394283100,2023-10-05 07:40:41.433055700,unl,informed5,6,4,8,2,2,4,0.25,0.5,2023-10-05 07:18:11.394283100,2023-10-05 07:33:11.433055700,1696490291,1696491191


In [47]:
data = pd.read_csv('./overhead.csv', header=0,  index_col=0)
final = pd.DataFrame(data)

final.head(10)

,identifier,topology,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,n_nodes,consensus,propagationTime,stddev_propagationTime,messageOverhead,stddev_messageOverhead
0,0,unl,8,6,12,4,8,2,0.25,1.0,24,3.022872,2149.216025,257.421035,1262081.5,255305.853095
1,3,unl,8,6,12,4,8,2,0.25,3.0,24,3.026471,2160.133107,126.586885,1220458.0,107468.917032
2,6,unl,16,8,20,16,8,4,0.50,1.0,24,3.027537,2008.636699,159.608146,991947.5,137230.334345
3,12,unl,16,8,20,16,8,4,0.50,0.5,24,3.073917,6674.953667,0.000000,388920.0,0.000000
4,13,unl,6,4,8,2,2,2,0.25,1.0,24,3.038808,3089.497825,259.143435,859426.5,79732.653540
5,18,unl,6,4,8,2,2,4,0.25,0.5,24,3.014395,2097.297225,201.000893,391164.5,201636.448403
6,21,unl,20,16,24,8,16,8,0.50,1.0,24,3.029904,2013.597379,287.651372,1063446.0,33021.886681
7,23,unl,16,8,20,4,8,2,0.50,1.0,24,3.021019,2353.420496,316.162061,1347907.5,95037.272712
8,27,unl,16,8,20,8,8,4,0.25,1.0,24,3.025704,1903.031597,0.000000,952511.0,0.000000
9,29,unl,16,8,20,4,8,2,0.25,1.0,24,3.044116,1963.333217,48.432417,809796.5,45515.756398


In [48]:
#Bandwidth
# recvRpc = pd.DataFrame()
# deliverMessage = pd.DataFrame()
# publishMessage = pd.DataFrame()

message = pd.DataFrame()

for index, row in final.iterrows():
    # print(row["start"], row["end"], row["experiment"])
    execs = exps.loc[(exps["topology"] == row["topology"]) & (exps["d"] == row["d"]) &
                            (exps["dlo"] == row["dlo"]) & (exps["dhi"] == row["dhi"]) &
                            (exps["dscore"] == row["dscore"]) & (exps["dlazy"] == row["dlazy"]) &
                            (exps["dout"] == row["dout"]) & (exps["gossipFactor"] == row["gossipFactor"]) &                    
                            (exps["interval"] == row["interval"])]

    for idx, exec in execs.iterrows():
        # print(exec["startUnix"], exec["endUnix"])
        query_mess = from_influx(url, token, org, "deliverMessage", exec["startUnix"], exec["endUnix"],"_measurement")
        query_rpc = from_influx(url, token, org, "recvRPC", exec["startUnix"], exec["endUnix"],"_measurement")
        # query_pub = from_influx(url, token, org, "publishMessage", exec["startUnix"], exec["endUnix"],"_measurement")
    
        query_mess["identifier"] = row["identifier"]
        query_rpc["identifier"] = row["identifier"]
        # query_pub["identifier"] = row["identifier"]

        query_mess["experiment"] = exec["experiment"]
        query_rpc["experiment"] = exec["experiment"]
        # query_pub["experiment"] = row["experiment"]
    
        query_mess["start"] = exec["start"]
        query_mess["end"] = exec["end"]
    
        query_rpc["start"] = exec["start"]
        query_rpc["end"] = exec["end"]
    
        # query_pub["start"] = exec["start"]
        # query_pub["end"] = exec["end"]
    
        # deliverMessage = pd.concat([query_mess, deliverMessage])
        # recvRpc = pd.concat([recvRpc, query_rpc])
        # publishMessage = pd.concat([publishMessage, query_pub])

        joined = pd.concat([query_rpc,query_mess])
        del query_rpc
        del query_mess
        # del deliverMessage
        gc.collect()
        
        joined["_time"] = pd.to_datetime(joined["_time"])
        message = pd.concat([message,joined])
        

message.head(10)


,_time,_measurement,identifier,experiment,start,end
0,2023-10-04 18:59:33.174112+00:00,recvRPC,0,390,2023-10-04 18:59:33.080773000,2023-10-04 19:14:33.086651800
1,2023-10-04 18:59:33.174543+00:00,recvRPC,0,390,2023-10-04 18:59:33.080773000,2023-10-04 19:14:33.086651800
2,2023-10-04 18:59:33.174555+00:00,recvRPC,0,390,2023-10-04 18:59:33.080773000,2023-10-04 19:14:33.086651800
3,2023-10-04 18:59:33.174764+00:00,recvRPC,0,390,2023-10-04 18:59:33.080773000,2023-10-04 19:14:33.086651800
4,2023-10-04 18:59:33.175009+00:00,recvRPC,0,390,2023-10-04 18:59:33.080773000,2023-10-04 19:14:33.086651800
5,2023-10-04 18:59:33.175069+00:00,recvRPC,0,390,2023-10-04 18:59:33.080773000,2023-10-04 19:14:33.086651800
6,2023-10-04 18:59:33.175210+00:00,recvRPC,0,390,2023-10-04 18:59:33.080773000,2023-10-04 19:14:33.086651800
7,2023-10-04 18:59:33.175240+00:00,recvRPC,0,390,2023-10-04 18:59:33.080773000,2023-10-04 19:14:33.086651800
8,2023-10-04 18:59:33.175278+00:00,recvRPC,0,390,2023-10-04 18:59:33.080773000,2023-10-04 19:14:33.086651800
9,2023-10-04 18:59:33.175305+00:00,recvRPC,0,390,2023-10-04 18:59:33.080773000,2023-10-04 19:14:33.086651800


In [49]:
#Try resampling for every seconds
dfNoIndex = message.reset_index()
# dfNoIndex.head(10)

by_time = dfNoIndex.groupby([dfNoIndex['experiment'],dfNoIndex["start"],dfNoIndex["end"],dfNoIndex["identifier"],pd.Grouper(key="_time", freq='1s')])["_measurement"].count().reset_index()
dfAggTime = by_time.rename(columns={"_measurement": "count"})

# date_list = pd.date_range(dfAggTime['start'].min(), dfAggTime['end'].max(),freq='1s',tz=None)

# dates = pd.DataFrame(date_list).rename(columns={0:"_time"})
# dates['count'] = 0

# dates['_time'] = pd.to_datetime(dates["_time"], format='mixed')#.tz_localize(None)
dfAggTime['_time'] = pd.to_datetime(dfAggTime["_time"], format='mixed')#.tz_localize(None)
dfAggTime['start'] = pd.to_datetime(dfAggTime["start"], format='mixed')#.tz_localize(None)
dfAggTime['end'] = pd.to_datetime(dfAggTime["end"], format='mixed')#.tz_localize(None)

dfAggTime["duration"] = (dfAggTime["end"]-dfAggTime["start"])/pd.Timedelta(seconds=1)
# dates["_time"] = dates["_time"].dt.tz_localize(None)
# dfAggTime["_time"] = dfAggTime["_time"].dt.tz_localize(None)
# dfAggTime["start"] = dfAggTime["start"].dt.tz_localize(None)
# dfAggTime["end"] = dfAggTime["end"].dt.tz_localize(None)

dfAggTime.head(10)

,experiment,start,end,identifier,_time,count,duration
0,390,2023-10-04 18:59:33.080773,2023-10-04 19:14:33.086651800,0,2023-10-04 18:59:33+00:00,130,900.005879
1,390,2023-10-04 18:59:33.080773,2023-10-04 19:14:33.086651800,0,2023-10-04 18:59:34+00:00,296,900.005879
2,390,2023-10-04 18:59:33.080773,2023-10-04 19:14:33.086651800,0,2023-10-04 18:59:35+00:00,2478,900.005879
3,390,2023-10-04 18:59:33.080773,2023-10-04 19:14:33.086651800,0,2023-10-04 18:59:36+00:00,423,900.005879
4,390,2023-10-04 18:59:33.080773,2023-10-04 19:14:33.086651800,0,2023-10-04 18:59:37+00:00,470,900.005879
5,390,2023-10-04 18:59:33.080773,2023-10-04 19:14:33.086651800,0,2023-10-04 18:59:38+00:00,2458,900.005879
6,390,2023-10-04 18:59:33.080773,2023-10-04 19:14:33.086651800,0,2023-10-04 18:59:39+00:00,131,900.005879
7,390,2023-10-04 18:59:33.080773,2023-10-04 19:14:33.086651800,0,2023-10-04 18:59:40+00:00,288,900.005879
8,390,2023-10-04 18:59:33.080773,2023-10-04 19:14:33.086651800,0,2023-10-04 18:59:41+00:00,2484,900.005879
9,390,2023-10-04 18:59:33.080773,2023-10-04 19:14:33.086651800,0,2023-10-04 18:59:42+00:00,415,900.005879


In [50]:
dfAggTime.tail(10)

,experiment,start,end,identifier,_time,count,duration
29537,499,2023-10-07 17:58:32.164634200,2023-10-07 18:13:32.168334600,85,2023-10-07 18:13:22+00:00,9,900.0037
29538,499,2023-10-07 17:58:32.164634200,2023-10-07 18:13:32.168334600,85,2023-10-07 18:13:23+00:00,800,900.0037
29539,499,2023-10-07 17:58:32.164634200,2023-10-07 18:13:32.168334600,85,2023-10-07 18:13:24+00:00,232,900.0037
29540,499,2023-10-07 17:58:32.164634200,2023-10-07 18:13:32.168334600,85,2023-10-07 18:13:25+00:00,9,900.0037
29541,499,2023-10-07 17:58:32.164634200,2023-10-07 18:13:32.168334600,85,2023-10-07 18:13:26+00:00,797,900.0037
29542,499,2023-10-07 17:58:32.164634200,2023-10-07 18:13:32.168334600,85,2023-10-07 18:13:27+00:00,233,900.0037
29543,499,2023-10-07 17:58:32.164634200,2023-10-07 18:13:32.168334600,85,2023-10-07 18:13:28+00:00,9,900.0037
29544,499,2023-10-07 17:58:32.164634200,2023-10-07 18:13:32.168334600,85,2023-10-07 18:13:29+00:00,798,900.0037
29545,499,2023-10-07 17:58:32.164634200,2023-10-07 18:13:32.168334600,85,2023-10-07 18:13:30+00:00,231,900.0037
29546,499,2023-10-07 17:58:32.164634200,2023-10-07 18:13:32.168334600,85,2023-10-07 18:13:31+00:00,9,900.0037


In [51]:
df=dfAggTime.drop(columns=["start", "end", "_time"])
avgPropExp = df.groupby(['experiment', 'identifier', 'duration']).agg('sum')
avgPropExp.reset_index(inplace=True)
avgPropExp['mean'] = avgPropExp['count']/avgPropExp['duration']

avgPropExp.head(10)

,experiment,identifier,duration,count,mean
0,390,0,900.005879,1875923,2084.345274
1,391,0,900.002246,1418974,1576.633844
2,393,3,900.005056,1549965,1722.173658
3,394,3,900.003317,1341154,1490.165620
4,396,6,900.002739,1069163,1187.955274
5,397,6,900.003133,1270404,1411.555086
6,404,12,900.052000,559415,621.536311
7,405,13,900.002258,1144174,1271.301256
8,406,13,900.002554,1308506,1453.891430
9,412,18,900.038773,364772,405.284762


In [54]:
avgProp = avgPropExp.drop(columns=['experiment', 'duration', 'count'])
avgBand = avgProp.groupby(['identifier']).agg({'mean':['mean','std']}).fillna(0)
avgBand.columns = avgBand.columns.droplevel(0)
avgBand.reset_index(inplace=True)

avgBand.head(10)

,identifier,mean,std
0,0,1830.489559,359.006195
1,3,1606.169639,164.054457
2,6,1299.755180,158.108944
3,12,621.536311,0.000000
4,13,1362.596343,129.110750
5,18,592.199166,264.336884
6,21,1429.535814,68.295928
7,23,1784.607281,125.436692
8,27,1281.485378,0.000000
9,29,1166.508678,67.757419


In [55]:
finalBandwidth = final.merge(avgBand, on='identifier')
finalBandwidth = finalBandwidth.rename(columns={'mean' : 'bandwidth', 'std' : 'stddev_bandwidth'})#.drop(columns=['std'])

finalBandwidth.to_csv('bandwidth.csv')
finalBandwidth.head(10)

,identifier,topology,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,n_nodes,consensus,propagationTime,stddev_propagationTime,messageOverhead,stddev_messageOverhead,bandwidth,stddev_bandwidth
0,0,unl,8,6,12,4,8,2,0.25,1.0,24,3.022872,2149.216025,257.421035,1262081.5,255305.853095,1830.489559,359.006195
1,3,unl,8,6,12,4,8,2,0.25,3.0,24,3.026471,2160.133107,126.586885,1220458.0,107468.917032,1606.169639,164.054457
2,6,unl,16,8,20,16,8,4,0.50,1.0,24,3.027537,2008.636699,159.608146,991947.5,137230.334345,1299.755180,158.108944
3,12,unl,16,8,20,16,8,4,0.50,0.5,24,3.073917,6674.953667,0.000000,388920.0,0.000000,621.536311,0.000000
4,13,unl,6,4,8,2,2,2,0.25,1.0,24,3.038808,3089.497825,259.143435,859426.5,79732.653540,1362.596343,129.110750
5,18,unl,6,4,8,2,2,4,0.25,0.5,24,3.014395,2097.297225,201.000893,391164.5,201636.448403,592.199166,264.336884
6,21,unl,20,16,24,8,16,8,0.50,1.0,24,3.029904,2013.597379,287.651372,1063446.0,33021.886681,1429.535814,68.295928
7,23,unl,16,8,20,4,8,2,0.50,1.0,24,3.021019,2353.420496,316.162061,1347907.5,95037.272712,1784.607281,125.436692
8,27,unl,16,8,20,8,8,4,0.25,1.0,24,3.025704,1903.031597,0.000000,952511.0,0.000000,1281.485378,0.000000
9,29,unl,16,8,20,4,8,2,0.25,1.0,24,3.044116,1963.333217,48.432417,809796.5,45515.756398,1166.508678,67.757419
